In [ ]:
1. 环境准备 : 导入各种需要的包
2. 数据准备与预处理
  2.1 定义数据转换
  2.2 加载数据集
  2.3 创建数据加载器
3. 模型构建
  3.1 自定义CNN模型
  3.2 使用预训练模型（可选）
4. 训练准备
  4.1 初始化模型
  4.2 定义损失函数和优化器
  4.3 训练和验证函数
5. 训练循环
6. 训练过程可视化
7. 模型评估
  7.1 加载最佳模型
  7.2 测试集评估
  7.3 混淆矩阵和分类报告
8. 预测单张图片
9. 模型部署（可选）

这个完整的PyTorch图片分类工作流程包括：
1.数据准备与增强
2.模型构建（自定义或预训练模型）
3.训练循环与验证
4.模型评估与可视化
5.预测功能
6.模型保存与部署

# PyTorch 图像分类模型示例，使用经典的 CIFAR-10 数据集

In [ ]:
# 导入必要的库
import torch
import torch.nn as nn        # 用于构建神经网络 的模块
import torch.optim as optim  # 用于构建优化器（优化函数）的模块
from torch.utils.data import DataLoader # 数据加载模块：它将数据集封装成一个可迭代对象，支持自动批处理、随机打乱、多进程并行加载等功能
import torchvision # 是 PyTorch 的官方视觉库，专门为计算机视觉任务设计，提供了三大核心功能：1预训练模型、2数据集、3数据变换
import torchvision.transforms as transforms # 数据变换模块：提供丰富的图像预处理和数据增强方法
import matplotlib.pyplot as plt
import numpy as np

# ================================
# 1. 设置设备：使用 GPU（如果可用）或 CPU
# ================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

# ================================
# 2. 数据预处理和加载
# ================================

# 定义图像预处理操作
# transforms.Compose 将多个操作组合在一起
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),        # 随机水平翻转，增强数据多样性
    transforms.RandomCrop(32, padding=4),     # 随机裁剪到 32x32，边缘填充 4 像素
    transforms.ToTensor(),                    # 转换为 Tensor (C, H, W)
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),  # 标准化：减均值除标准差
                         std=(0.2023, 0.1994, 0.2010))   # CIFAR-10 的均值和标准差
])

# 测试集只做标准化，不做数据增强
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),
                         std=(0.2023, 0.1994, 0.2010))
])

# 加载 CIFAR-10 训练集
train_dataset = torchvision.datasets.CIFAR10(
    root='./data',              # 数据保存路径
    train=True,                 # 使用训练集
    download=True,              # 如果本地没有，自动下载
    transform=transform_train   # 应用预处理
)

# 加载测试集
test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,                # 使用测试集
    download=True,
    transform=transform_test
)

# 创建数据加载器（DataLoader），用于批量读取数据
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=128,             # 每批 128 张图片
    shuffle=True,               # 打乱顺序
    num_workers=2               # 使用 2 个子进程加载数据（加快速度）
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=100,             # 测试时 batch 可稍大
    shuffle=False,
    num_workers=2
)

# CIFAR-10 的类别名称
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

# ================================
# 3. 定义卷积神经网络模型
# ================================

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        # 第一个卷积层：3通道输入 -> 32 输出通道，卷积核 3x3
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)  # 2x2 最大池化，尺寸减半

        # 第二个卷积层：32 -> 64 通道
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)

        # 第三个卷积层：64 -> 128 通道
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2)  # 经过三次池化，32 -> 4

        # 全连接层：将特征图展平后分类
        self.fc1 = nn.Linear(128 * 4 * 4, 512)  # 4x4 是最后特征图大小
        self.relu4 = nn.ReLU()
        self.dropout = nn.Dropout(0.5)  # 防止过拟合
        self.fc2 = nn.Linear(512, num_classes)  # 输出 10 类

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.pool3(self.relu3(self.conv3(x)))
        x = x.view(x.size(0), -1)  # 展平成 (batch_size, 128*4*4)
        x = self.relu4(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# 实例化模型并移动到设备（GPU/CPU）
model = SimpleCNN(num_classes=10).to(device)

# ================================
# 4. 定义损失函数和优化器
# ================================

criterion = nn.CrossEntropyLoss()  # 多分类交叉熵损失
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 使用 Adam 优化器

# ================================
# 5. 训练模型
# ================================

num_epochs = 10
train_losses = []
test_accuracies = []

for epoch in range(num_epochs):
    model.train()  # 切换到训练模式（启用 Dropout 等）
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # 移动到 GPU

        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()  # 清除梯度
        loss.backward()        # 反向传播
        optimizer.step()       # 更新参数

        running_loss += loss.item()

    # 计算平均训练损失
    avg_loss = running_loss / len(train_loader)
    train_losses.append(avg_loss)

    # 每个 epoch 后在测试集上评估
    model.eval()  # 切换到评估模式（关闭 Dropout）
    correct = 0
    total = 0
    with torch.no_grad():  # 不计算梯度，节省内存
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100. * correct / total
    test_accuracies.append(accuracy)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')

# ================================
# 6. 绘制训练过程曲线
# ================================

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(test_accuracies, color='orange', label='Test Accuracy')
plt.title('Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.tight_layout()
plt.show()

# ================================
# 7. 保存模型
# ================================

torch.save(model.state_dict(), 'cifar10_cnn_model.pth')
print("模型已保存: cifar10_cnn_model.pth")

# ================================
# 8. 可选：测试单张图片并可视化
# ================================

def imshow(img):
    img = img * torch.tensor([0.2023, 0.1994, 0.2010]).view(3,1,1) + \
          torch.tensor([0.4914, 0.4822, 0.4465]).view(3,1,1)  # 反标准化
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis('off')
    plt.show()

# 获取一批测试图像
dataiter = iter(test_loader)
images, labels = next(dataiter)
images, labels = images[:4], labels[:4]  # 取前 4 张

# 移动到设备
images, labels = images.to(device), labels.to(device)

# 预测
model.eval()
with torch.no_grad():
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)

# 显示图像和预测结果
imshow(torchvision.utils.make_grid(images.cpu()))
print('真实标签:', ' '.join(f'{classes[labels[i]]}' for i in range(4)))
print('预测标签:', ' '.join(f'{classes[predicted[i]]}' for i in range(4)))

# PyTorch 回归模型示例

In [ ]:
# 导入必要的库
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset

# ================================
# 1. 设置设备：使用 GPU（如果可用）或 CPU
# ================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

# ================================
# 2. 生成模拟回归数据
# ================================
# 我们模拟一个非线性关系：y = x^2 + sin(x) + 噪声
np.random.seed(42)
torch.manual_seed(42)

# 生成 1000 个数据点，x 在 [-5, 5] 区间
x = np.random.uniform(-5, 5, size=(1000, 1))
y = x ** 2 + np.sin(x * 2) + np.random.normal(0, 0.5, x.shape)  # 添加噪声

# 转换为 PyTorch 的 Tensor
x_tensor = torch.FloatTensor(x).to(device)
y_tensor = torch.FloatTensor(y).to(device)

# ================================
# 3. 创建数据集和数据加载器
# ================================
# 使用 TensorDataset 将输入和标签打包
dataset = TensorDataset(x_tensor, y_tensor)

# 划分训练集和测试集（80% 训练，20% 测试）
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# 创建 DataLoader，支持批量训练
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"训练样本数: {len(train_dataset)}")
print(f"测试样本数: {len(test_dataset)}")

# ================================
# 4. 定义回归模型（多层感知机 MLP）
# ================================
class RegressionModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, output_size=1):
        super(RegressionModel, self).__init__()
        # 定义网络结构：输入层 -> 隐藏层 -> 激活函数 -> 隐藏层 -> 激活函数 -> 输出层
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),    # 第一层：1 -> 64
            nn.ReLU(),                             # 激活函数
            nn.Linear(hidden_size, hidden_size),   # 第二层：64 -> 64
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)    # 输出层：64 -> 1（回归输出单值）
        )

    def forward(self, x):
        return self.network(x)

# 实例化模型并移动到设备
model = RegressionModel(input_size=1, hidden_size=64, output_size=1).to(device)

# ================================
# 5. 定义损失函数和优化器
# ================================
# 回归任务使用均方误差（MSE）
criterion = nn.MSELoss()

# 使用 Adam 优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ================================
# 6. 训练模型
# ================================
num_epochs = 200
train_losses = []

for epoch in range(num_epochs):
    model.train()  # 切换到训练模式（虽然这里没有 Dropout/BatchNorm）
    running_loss = 0.0

    for inputs, targets in train_loader:
        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # 反向传播
        optimizer.zero_grad()  # 清除梯度
        loss.backward()        # 反向传播计算梯度
        optimizer.step()       # 更新参数

        running_loss += loss.item()

    # 记录每个 epoch 的平均损失
    avg_loss = running_loss / len(train_loader)
    train_losses.append(avg_loss)

    # 每 20 个 epoch 打印一次
    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.6f}')

# ================================
# 7. 绘制训练损失曲线
# ================================
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.legend()
plt.grid(True)

# ================================
# 8. 在测试集上评估并可视化预测结果
# ================================
model.eval()  # 切换到评估模式
test_predictions = []
test_targets = []

with torch.no_grad():  # 不需要梯度计算
    for inputs, targets in test_loader:
        outputs = model(inputs)
        test_predictions.append(outputs.cpu().numpy())
        test_targets.append(targets.cpu().numpy())

# 合并所有 batch 的结果
test_predictions = np.concatenate(test_predictions)
test_targets = np.concatenate(test_targets)

# 计算测试集 MSE 和 R²（决定系数）
from sklearn.metrics import r2_score
test_mse = np.mean((test_predictions - test_targets) ** 2)
test_r2 = r2_score(test_targets, test_predictions)

print(f"\n测试集 MSE: {test_mse:.6f}")
print(f"测试集 R²: {test_r2:.4f}")

# ================================
# 9. 可视化真实值 vs 预测值
# ================================
# 为了可视化，我们按 x 排序
# 获取所有测试数据的输入 x
x_test = np.concatenate([x.cpu().numpy() for x, _ in test_loader])
idx_sorted = np.argsort(x_test.flatten())

x_sorted = x_test[idx_sorted]
y_true_sorted = test_targets[idx_sorted]
y_pred_sorted = test_predictions[idx_sorted]

plt.subplot(1, 2, 2)
plt.scatter(x_test, test_targets, alpha=0.5, label='真实值', s=10)
plt.plot(x_sorted, y_pred_sorted, color='red', linewidth=2, label='模型预测')
plt.title('真实值 vs 模型预测')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# ================================
# 10. 保存模型
# ================================
torch.save(model.state_dict(), 'regression_model.pth')
print("模型已保存: regression_model.pth")

# ================================
# 11. 使用模型进行单个预测（推理）
# ================================
# 示例：预测 x = 2.5
model.eval()
x_new = torch.FloatTensor([[2.5]]).to(device)
with torch.no_grad():
    y_pred = model(x_new).cpu().numpy()

print(f"输入 x = 2.5，模型预测 y = {y_pred[0][0]:.4f}")